In [1]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
import time
import matplotlib.pyplot as plt
from fceu_help import *
from math import *
import pickle

In [2]:
# Needed for Joe's pyenv to find CV2
import site
site.addsitedir("/usr/local/lib/python2.7/site-packages")

In [3]:

START = 0x08
RUN = 0x02
JUMP = 0x01

#inputVec = fceulib.readInputs('videos/algar-batman.fm2')
#emu = fceulib.runGame('batman.nes')




In [6]:
import cv2 
from scipy import stats
from PIL import Image
inputVec = fceulib.readInputs('Illustrative.fm2') #'videos/happylee4-smb-warpless.fm2')
emu = fceulib.runGame('mario.nes')

image_prefix = 'images/mario'
nt_prefix = 'nametables'
#inputVec = fceulib.readInputs('videos/Contra by 3D.fm2')
#emu = fceulib.runGame('contra.nes')

start = VectorBytes()
emu.save(start)


peekevery = 1
emu.load(start)
display = False
img_buffer = VectorBytes()
prev = VectorBytes()

display = False
start = 270
end = float('inf')
colorized2id = {}
id2colorized = {}
outfile = open('Illustrative.txt','wb')
nametables = {}
nametables2 = {}
xScrolls = None
motion = {}
tnametables = {}
for timestep, inp in enumerate(inputVec):
    emu.stepFull(inp,0x0)
    if not (timestep % peekevery == 0) or timestep < start  :
        continue
    if timestep > end:
        break
    if timestep % 100 == 0:
        print (100*float(timestep)/float(len(inputVec)))
        
    if xScrolls is not None:
        xScrolls[timestep]  = emu.xScroll
    nt_index = pointer_to_numpy(emu.fc.ppu.values)[0]& 0x3    
    if nametables is not None:
        nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
        attr = nt[(nt_index*1024+960):(nt_index*1024+1024)]
        nt = nt[(nt_index*1024):(nt_index*1024+960)]
        nt = nt.reshape(30,32)
        attr = attr.reshape(8,8)
        full_attr = np.zeros([16,16])
        for xx in range(8):
            for yy in range(8):
                val = attr[yy,xx]
                tr = (val & 0x0c) >> 2
                tl = val & 0x03
                br = (val & 0xc0 ) >> 6
                bl = (val & 0x30 ) >> 4
                full_attr[yy*2,xx*2] = tl
                full_attr[yy*2,xx*2+1] = tr
                full_attr[yy*2+1,xx*2] = bl
                full_attr[yy*2+1,xx*2+1] = br
        full_attr = full_attr[:15,:]
        full_attr = np.kron(full_attr, np.ones((2,2)))
        nametables = (nt,full_attr)
    nt_index = 1 - nt_index
    if nametables2 is not None:
        nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
        attr = nt[(nt_index*1024+960):(nt_index*1024+1024)]
        nt = nt[(nt_index*1024):(nt_index*1024+960)]
        nt = nt.reshape(30,32)
        attr = attr.reshape(8,8)
        full_attr = np.zeros([16,16])
        for xx in range(8):
            for yy in range(8):
                val = attr[yy,xx]
                tr = (val & 0x0c) >> 2
                tl = val & 0x03
                br = (val & 0xc0 ) >> 6
                bl = (val & 0x30 ) >> 4
                full_attr[yy*2,xx*2] = tl
                full_attr[yy*2,xx*2+1] = tr
                full_attr[yy*2+1,xx*2] = bl
                full_attr[yy*2+1,xx*2+1] = br
        full_attr = full_attr[:15,:]
        full_attr = np.kron(full_attr, np.ones((2,2)))
        nametables2 = (nt,full_attr)
    
    xScroll = emu.fc.ppu.xScroll
    yScroll = emu.fc.ppu.yScroll
    fineXScroll = xScroll & 0x7
    coarseXScroll = xScroll >> 3
    NT = np.kron(nametables[0], np.ones((8,8)))
    NT2 = np.kron(nametables2[0], np.ones((8,8)))
    actualNT = np.zeros(NT.shape)
    actualNT[:,:(NT.shape[1]-xScroll)] = NT[:,xScroll:]
    actualNT[:,(NT.shape[1]-xScroll):] = NT2[:,:xScroll]
    
    im = Image.fromarray(actualNT).convert('RGB')
    im.save(nt_prefix + '/nametable_{}.png'.format(timestep))
    
    attr = np.kron(nametables[1], np.ones((8,8)))
    attr2 = np.kron(nametables2[1], np.ones((8,8)))

    actualattr = np.zeros(NT.shape)
    actualattr[:,:(attr.shape[1]-xScroll)] = attr[:,xScroll:]
    actualattr[:,(attr.shape[1]-xScroll):] = attr2[:,:xScroll]
    #tnametables[timestep] = (actualNT,actualattr)
    
    im = Image.fromarray(actualattr).convert('RGB')
    im.save(nt_prefix + '/attributes_{}.png'.format(timestep))
    
    if display:
        outputImage(emu, '{}_{}'.format(image_prefix,timestep),img_buffer)
    sprite_list,colorized_sprites = get_all_sprites(emu.fc)  
    
    for sprite_id,sprite in enumerate(sprite_list):
        if np.sum(colorized_sprites[sprite_id].ravel()) == 0:
            continue
        uniq = tuple(colorized_sprites[sprite_id].ravel())
        if uniq not in colorized2id:
            colorized2id[uniq] = len(colorized2id)
            print len(colorized2id)
            id2colorized[colorized2id[uniq]] = colorized_sprites[sprite_id]
        outfile.write('{},{},{},{}\n'.format(timestep,colorized2id[uniq],sprite[0],sprite[1]))

outfile.close()
if id2colorized:
    pickle.dump((colorized2id,id2colorized),open('id2sprites.pkl','wb'))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
4.08608008717
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
5.44810678289
6.81013347862
8.17216017434
9.53418687006
36
37
38
39
40
41
10.8962135658
42
43
44
45
12.2582402615
46
47
48
49
13.6202669572
50
51
52
53
54
55
56
57
58
59
60
61
62
14.982293653
16.3443203487
17.7063470444
63
64
19.0683737401
20.4304004358
21.7924271316
23.1544538273
24.516480523
25.8785072187
27.2405339145
28.6025606102
29.9645873059
31.3266140016
32.6886406974
34.0506673931
35.4126940888
36.7747207845
38.1367474803
39.498774176
40.8608008717
42.2228275674
43.5848542631
44.9468809589
46.3089076546
47.6709343503
65
66
67
68
49.032961046
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
50.3949877418
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
51.7570144375
53.1190411332
54.4810678289
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
55.8430945247
134
135
136
137
138
139
140
141
57

In [7]:


#uses normalized pmi as a thresholding mechanism 
# -1 = no co-occurences, 
# 0 = independent 
# 1 = only co-occur
def getFusedPairs(filename,fc,threshold=0.1):
    #Read in data
    with open(filename) as infile:
        data = []
        for line in infile:
            data.append([int(float(d)) for d in line.rstrip().split(',')])
    #get the per timestep data, as well as total counts for each sprite
    timesteps = {}
    pSprite = {}
    for d in data:
        if d[0] not in timesteps:
            timesteps[d[0]] = []
        timesteps[d[0]].append(d[1:])
        if d[1] not in pSprite:
            pSprite[d[1]] = 0
        pSprite[d[1]] += 1
    spr = get_sprite(0,emu.fc)
    height = spr.shape[0]
    #for each timestep look at all the sprite pairs and see how far apart they are,
    #if they are within width and height of each other, add that as a co-occurence
    
    all_sprites = {}
    for timestep in sorted(timesteps):
        dat = timesteps[timestep]
        for ii in range(len(dat)):
            for jj in range(len(dat)):
                if ii != jj:
                    id1 = dat[ii][0]
                    id2 = dat[jj][0]
                    dx = dat[ii][1]-dat[jj][1]
                    dy = dat[ii][2]-dat[jj][2]
                    
                    if abs(dx) > 8 or abs(dy) > height:
                        continue
                    if id1 not in all_sprites:
                        all_sprites[id1] = {}
                    if id2 not in all_sprites[id1]:
                        all_sprites[id1][id2] = 0
                    all_sprites[id1][id2] += 1
    
    #now we have all of the pieces and can calculate the pmi for each pair of sprites
    accepted = set()
    for sprite in all_sprites:
        total = 0
        dat = []
        px = float(pSprite[sprite])/float(len(timesteps))

        for other in all_sprites[sprite]:
            py = float(pSprite[other])/float(len(timesteps))
            pxy = float(all_sprites[sprite][other])/float(len(timesteps))
            d = log(px*py)/log(pxy)-1
            if d > threshold:
                s = sprite
                o = other
                if sprite > other:
                    o = sprite
                    s = other
                accepted.add((s,o))
    return accepted
getFusedPairs('test.txt',emu.fc)

set()

In [ ]:
help(fceulib)

In [8]:
pickle.dump(motion,open('mario_screen_motion.pkl','wb'))